In [1]:
from operator import getitem
import numpy as np


import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

### Download fruit dataset from Kaggle

In [5]:
import kagglehub
path = kagglehub.dataset_download("chrisfilo/fruit-recognition")

### Preparing dataframe with image paths and labels

In [3]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [13]:
full_dataset = datasets.ImageFolder(path, transform=data_transforms)

targets = full_dataset.targets
train_idx, test_idx = train_test_split(
    np.arange(len(targets)),    # Split indices [0, 1, 2...]
    test_size=0.2,              # 20% for testing
    shuffle=True,
    stratify=targets,           # <--- KEY: Ensures equal proportion of fruits
    random_state=42
)
train_dataset = Subset(full_dataset, train_idx)
test_dataset = Subset(full_dataset, test_idx)

dataloaders = {
    'train': DataLoader(train_dataset, batch_size=10, shuffle=True, num_workers=2),
    'test': DataLoader(test_dataset, batch_size=10, shuffle=False, num_workers=2)
}

dataset_sizes = {'train': len(train_dataset), 'test': len(test_dataset)}

### ResNet18 model for classification

In [9]:
from torchvision import models
model = models.resnet18(weights='IMAGENET1K_V1')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Piotr Matusiewicz/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:04<00:00, 10.1MB/s]


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [12]:
for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
class_count = len(full_dataset.classes)
model.fc = nn.Linear(num_ftrs, class_count)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [18]:
import time
import copy

def train_model(model, criterion, optimizer, num_epochs=5):
    model.to(device)
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # Ustawiamy co ile batchy mamy drukować info (np. co 20)
    print_every = 20

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Dodaliśmy enumerate, żeby mieć licznik 'i' (numer batcha)
            for i, (inputs, labels) in enumerate(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statystyki
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                # --- NOWOŚĆ: DRUKOWANIE POSTĘPU ---
                # Drukujemy tylko w fazie treningu, co 'print_every' kroków
                if phase == 'train' and i % print_every == 0:
                    # Obliczamy średnią stratę do tego momentu
                    temp_loss = running_loss / ((i + 1) * inputs.size(0))
                    # Obliczamy ile zdjęć już przerobiliśmy
                    processed_imgs = (i + 1) * inputs.size(0)
                    total_imgs = dataset_sizes[phase]

                    print(f"  [Batch {i}] Przetworzono: {processed_imgs}/{total_imgs} | Loss: {temp_loss:.4f}")
                # ----------------------------------

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Total Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print(f'\nTraining complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    model.load_state_dict(best_model_wts)
    return model

In [18]:
model_ft = train_model(model, criterion, optimizer, num_epochs=5)


Epoch 1/5
----------
  [Batch 0] Przetworzono: 10/56439 | Loss: 0.0816
  [Batch 20] Przetworzono: 210/56439 | Loss: 0.1446


KeyboardInterrupt: 


KeyboardInterrupt



device(type='cpu')